In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/sentiment_data/')

In [ ]:
!pip install emot
!pip install emoji
!git clone https://github.com/aub-mind/arabert.git
!pip install pyarabic
!pip install arabert
!pip install -U huggingface_hub
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=02354a2869e63f3f14cdc3dfa3a58656d24cb36168de0e4dce3111e8dbf4aea7
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 22.78 MiB/s, done.
Resolving deltas: 100

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt 
import seaborn as sns
from emot.emo_unicode import UNICODE_EMOJI ,EMOJI_UNICODE, EMOTICONS_EMO
import re
import string
from arabert.preprocess import ArabertPreprocessor
import emoji


In [ ]:
neg_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment_data/test_Arabic_tweets_negative_20190413.tsv",sep='\t',header=None)
pos_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment_data/test_Arabic_tweets_positive_20190413.tsv",sep='\t',header=None)

In [ ]:
df = pd.concat([neg_df,pos_df])

In [ ]:
df = df.sample(frac=1)

In [ ]:
df.head()

,0,1
1282,pos,هدف في وقته برافو بوتيا 💙
4039,neg,هه 😃 😃 ايه ياغادة. وجعلناكم شعوبا وقبائل لتعار...
1454,pos,فلت بوتات على كيف كيفك 😘
4687,pos,بمناسبة فوز الهلال .. 💙 سحب على آيفون XR📱 رتوي...
4895,neg,والاهم من كل ده ويحزن الواحد اقسم بالله يا عم ...


data cleaning

remove emoji

In [ ]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text
# Example
print('before Remove emoji')
print(df.iloc[387,1])
print('After Remove emoji')
convert_emojis(df.iloc[387,1][1:-1])

before Remove emoji
مثانه بوليه رياكتد هاها
After Remove emoji


'ثانه بوليه رياكتد هاه'

remove emojiicon :-)

In [ ]:
# Function for converting emoticons into word
def convert_emoticons(text):
    #text = re.escape(text)
    for emot in EMOTICONS_EMO:

        text = text.replace(emot, "_".join(EMOTICONS_EMO[emot].replace(",","").split()))
    return text
# Example
print('before Remove emoji')
print(df.iloc[109,1])
print('After Remove emoji')
convert_emoticons(df.iloc[109,1][1:-1])

before Remove emoji
تم 👌 تابعني 💙 اتابعك 💙
After Remove emoji


'م 👌 تابعني 💙 اتابعك '

second function

In [ ]:

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

In [ ]:
give_emoji_free_text("تعالو 😴 ! #طفش")

'تعالو  ! #طفش'

In [ ]:
def data_cleaning (text):
    # clean-up:
    # remove qutoions 
    text = text.strip()

    text= re.sub(r'http\S+', '', text)
    
    text= re.sub(r'ي+','ي', text)
    text = text.replace("آ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")


    text= re.sub(r'[@|#]\S*', '',text)
    text= re.sub(r'"+', '', text)
    # Remove arabic signs
    text= re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', text)
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
 
   
    

    # Remove repeated letters like "الللللللللللللللله" to "الله"
    text= text[0:2] + ''.join([text[i] for i in range(2, len(text)) if text[i]!=text[i-1] or text[i]!=text[i-2]])
    text= re.sub(r'D',':D', text)
    text= re.sub(r'هه+', 'face_with_tears_of_joy', text)
    text= convert_emojis(text)
    text= convert_emoticons(text)
    text = give_emoji_free_text(text)

     # Removing punctuations in string
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    text=re.sub(r'(?:^| )\w(?:$| )', ' ', text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub(" \d+", " ", text)
    text = re.sub("(\s\d+)","",text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub("(\s\d+)","",text) 
    # Returns: hi what is the weather like
    

    return text

In [ ]:
model_name="aubmindlab/bert-base-arabertv02-twitter"
arabert_prep = ArabertPreprocessor(model_name=model_name)

In [ ]:
df['preprocessing'] =df.iloc[:,1].apply(lambda x:data_cleaning(x))
df['preprocessing'] =df['preprocessing'] .apply(lambda x:arabert_prep.preprocess(x))

In [ ]:
len(df.index)

11520

In [ ]:
import numpy as np

df['preprocessing'] = df['preprocessing'].replace('', np.nan)
df = df.dropna(axis=0, subset=['preprocessing'])

In [ ]:
len(df.index)

11506

In [ ]:
df.values.tolist()[:3]

[['pos', 'هدف في وقته برافو بوتيا 💙', 'هدف في وقته برافو بوتيا'],
 ['neg',
  'هه 😃 😃 ايه ياغادة. وجعلناكم شعوبا وقبائل لتعارفوا 🙈 🙈 🙈 والنبي وصي علي سابع جاار 🤭🤭',
  'facewithtearsofjoy ايه ياغادة وجعلناكم شعوبا وقبايل لتعارفوا والنبي وصي علي سابع جاار'],
 ['pos', 'فلت بوتات على كيف كيفك 😘', 'فلت بوتات على كيف كيفك']]

In [ ]:
# df.values.tolist()

In [ ]:
from transformers import pipeline
sa = pipeline('text-classification', model='CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment')
sentences = ['صباح الياسمين']
sent=sa(sentences)
print(sent[0]['label'])
print(sent[0]['score'])



Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


positive
0.9452193379402161


In [ ]:
df.values.tolist()[:1]

[['pos', 'هدف في وقته برافو بوتيا 💙', 'هدف في وقته برافو بوتيا']]

In [ ]:
print(df.preprocessing.str.len().max())

3350


In [ ]:
# try:
#     something()
# except Exception as e:
#     send_somewhere(traceback.format_exception(*sys.exc_info()))

In [ ]:
# itemlist=[]
# list_of_list =[]
# for item in df.values.tolist():
#   try:
#     sent=sa(item[2])
#   except Exception:
#     print("")
#   else:

#     list_of_list.append([item[0],item[1],item[2],sent[0]['label'],sent[0]['score']])  
    

# list_of_list[:3]

KeyboardInterrupt: ignored

In [ ]:
# df_out = pd.DataFrame(list_of_list, columns = ['sent', 'text','proccessing','model_sent','model_score'])


In [ ]:
# df_out.to_csv('/content/drive/MyDrive/Colab Notebooks/sentiment_data/sentiment_out.csv',encoding='utf-8-sig',index=False)

In [ ]:
# df_out=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment_data/sentiment_out.csv',encoding='utf-8')
# df_out.head()

,sent,text,proccessing,model_sent,model_score
0,pos,اي وش بعد 😊,اي وش بعد,negative,0.725706
1,pos,[ 📷 ] • جيمين من جلسة تصوير ديسباتش ونايفر لأل...,جيمين من جلسة تصوير ديسباتش ونايفر لالبوم,neutral,0.960727
2,pos,. . ﷽ ﴿ قل هو الله أحد ۝ الله الصمد ۝ لم يلد و...,قل هو الله احد الله الصمد لم يلد ولم,neutral,0.573697
3,pos,"🎥 المهمة الأولى في ""جدة"" ✔💪🏼 💙 #الهلال #فيديو_...",المهمة الاولى في جدة,neutral,0.983851
4,neg,عييب هذا الكلاام 👎,عيب هذا الكلاام,negative,0.991403


In [ ]:
# df_out['model_sent']=df_out['model_sent'].apply(lambda x: x[:3])

In [ ]:
# df_out.head(20)

,sent,text,proccessing,model_sent,model_score
0,pos,اي وش بعد 😊,اي وش بعد,neg,0.725706
1,pos,[ 📷 ] • جيمين من جلسة تصوير ديسباتش ونايفر لأل...,جيمين من جلسة تصوير ديسباتش ونايفر لالبوم,neu,0.960727
2,pos,. . ﷽ ﴿ قل هو الله أحد ۝ الله الصمد ۝ لم يلد و...,قل هو الله احد الله الصمد لم يلد ولم,neu,0.573697
3,pos,"🎥 المهمة الأولى في ""جدة"" ✔💪🏼 💙 #الهلال #فيديو_...",المهمة الاولى في جدة,neu,0.983851
4,neg,عييب هذا الكلاام 👎,عيب هذا الكلاام,neg,0.991403
5,pos,الاهتمام لايگلف شيء لكنه يعني الكثير ❀,الاهتمام لاي لف شيء لكنه يعني الكثير,neu,0.655389
6,pos,وبعده الشعب القطري مغيب 🙂 ويقولون السعوديه وال...,وبعده الشعب القطري مغيب ويقولون السعوديه والام...,neg,0.995643
7,pos,طريقة تعارفهم لطيفة 😂,طريقة تعارفهم لطيفة,pos,0.895035
8,neg,اياك ان تؤذي نفسآ ، حتى ولو لم تكن من البشر 💔 ..,اياك ان توذي نفسا حتى ولو لم تكن من البشر,neg,0.413690
9,pos,لاتنسوا وردكم من القرآن,لاتنسوا وردكم من القران,pos,0.674752


In [ ]:
def complex_function(x):
    if x =='neu':
        return 'pos'
    else:
      return x

In [ ]:
# df_out['model_sent']=df_out['model_sent'].apply(lambda x:complex_function(x))
# df_out.head()

,sent,text,proccessing,model_sent,model_score
0,pos,اي وش بعد 😊,اي وش بعد,neg,0.725706
1,pos,[ 📷 ] • جيمين من جلسة تصوير ديسباتش ونايفر لأل...,جيمين من جلسة تصوير ديسباتش ونايفر لالبوم,pos,0.960727
2,pos,. . ﷽ ﴿ قل هو الله أحد ۝ الله الصمد ۝ لم يلد و...,قل هو الله احد الله الصمد لم يلد ولم,pos,0.573697
3,pos,"🎥 المهمة الأولى في ""جدة"" ✔💪🏼 💙 #الهلال #فيديو_...",المهمة الاولى في جدة,pos,0.983851
4,neg,عييب هذا الكلاام 👎,عيب هذا الكلاام,neg,0.991403


In [ ]:
len(list_of_list)

In [ ]:
# for array operations
import numpy as np 
# TensorFlow framework
import tensorflow as tf
# PyTorch framework
import torch
# for pretty printing
from pprint import pprint

In [ ]:
# tf.executing_eagerly()

In [ ]:
# checkpoint = 'CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment'
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# # build a tokenizer
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# # convert raw text into number inputs
# inputs = tokenizer(df['preprocessing'].values.tolist(), truncation=True, padding=True, return_tensors='tf')
# # pprint(inputs) 

In [ ]:
# inputs=tokenizer.batch_encode_plus(     df.preprocessing.values.tolist(),      add_special_tokens=True,      return_attention_mask=True,      padding='longest',     max_length=1024,     return_tensors='pt' )

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# # make predictions
# outputs = model(**inputs)
# outputs

In [ ]:
# sa(df['preprocessing'].values.tolist())

sentiment without preprocessing 

In [ ]:
# itemlist=[]
# list_of_list =[]
# for item in df.values.tolist():
#   try:
#     sent=sa(item[1])
#   except Exception:
#     print("")
#   else:

#     list_of_list.append([item[0],item[1],item[2],sent[0]['label'],sent[0]['score']])  

In [ ]:
# second_df = pd.DataFrame(list_of_list, columns = ['sent', 'text','proccessing','model_sent','model_score'])

In [ ]:
# second_df.to_csv('/content/drive/MyDrive/Colab Notebooks/sentiment_data/second_sentiment_out.csv',encoding='utf-8-sig',index=False)

In [ ]:
second_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment_data/second_sentiment_out.csv',encoding='utf-8')
second_df.head()

,sent,text,proccessing,model_sent,model_score
0,neg,📌 فيفا تركت الاتحادات والبطولات القارية وتركت ...,فيفا تركت الاتحادات والبطولات القارية وتركت ال...,neutral,0.866322
1,neg,❗ . . . الماضي للعبرة : من أدام النظر في صفحات...,الماضي للعبرة من ادام النظر في صفحات ماضيه لن ...,neutral,0.730296
2,neg,صدقت الحين 💔,صدقت الحين,positive,0.559969
3,pos,انت غير الناس عندي انت عندي شيء كبير 🎶,انت غير الناس عندي انت عندي شيء كبير,positive,0.845273
4,pos,كائن صباحي 😂,كاين صباحي,positive,0.893599


In [ ]:
second_df['model_sent']=second_df['model_sent'].apply(lambda x: x[:3])
second_df.head()

,sent,text,proccessing,model_sent,model_score
0,neg,📌 فيفا تركت الاتحادات والبطولات القارية وتركت ...,فيفا تركت الاتحادات والبطولات القارية وتركت ال...,neu,0.866322
1,neg,❗ . . . الماضي للعبرة : من أدام النظر في صفحات...,الماضي للعبرة من ادام النظر في صفحات ماضيه لن ...,neu,0.730296
2,neg,صدقت الحين 💔,صدقت الحين,pos,0.559969
3,pos,انت غير الناس عندي انت عندي شيء كبير 🎶,انت غير الناس عندي انت عندي شيء كبير,pos,0.845273
4,pos,كائن صباحي 😂,كاين صباحي,pos,0.893599


In [ ]:
# second_df['model_sent']=second_df['model_sent'].apply(lambda x:complex_function(x))
# second_df.head()

In [ ]:
second_df.drop(second_df.loc[second_df['model_sent']=='neu'].index, inplace=True)
second_df.head()

,sent,text,proccessing,model_sent,model_score
2,neg,صدقت الحين 💔,صدقت الحين,pos,0.559969
3,pos,انت غير الناس عندي انت عندي شيء كبير 🎶,انت غير الناس عندي انت عندي شيء كبير,pos,0.845273
4,pos,كائن صباحي 😂,كاين صباحي,pos,0.893599
5,pos,🍃🌸 گوني غير الگل 😌 .. تميزي بضحگتگ 🙊 . . بلمعة...,وني غير ال ل تميزي بضح ت بلمعة عيوون ببساطت بط...,pos,0.991739
6,neg,ثم يقولون ليش تخاف من البسابه 😑,ثم يقولون ليش تخاف من البسابه,neg,0.975234


In [ ]:
len(second_df.index)

9397

In [ ]:
second_df.to_csv('/content/drive/MyDrive/Colab Notebooks/sentiment_data/result_second_sentiment_out.csv',encoding='utf-8-sig',index=False)